In [1]:
import tensorflow as tf
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

## 一．张量数据结构
分为常量constant和变量Variable

### 1.常量张量
在计算图中不可以被重新赋值

In [2]:
tf.constant(1)
tf.constant(1,dtype=tf.int64)
tf.constant(1.23)
tf.constant(3.14,dtype=tf.float64)
tf.constant('helloworld')
tf.constant(True)

<tf.Tensor: shape=(), dtype=int32, numpy=1>

<tf.Tensor: shape=(), dtype=int64, numpy=1>

<tf.Tensor: shape=(), dtype=float32, numpy=1.23>

<tf.Tensor: shape=(), dtype=float64, numpy=3.14>

<tf.Tensor: shape=(), dtype=string, numpy=b'helloworld'>

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [3]:
tf.int64 == np.int64
tf.bool == np.bool
tf.double == np.float64
tf.string == np.unicode

True

True

True

False

记：几层中括号，即为多少维的张量
tf.rank(v)和numpy的ndim方法相同

In [4]:
# 标量
scalar = tf.constant(True)
tf.rank(scalar)
scalar.numpy().ndim

<tf.Tensor: shape=(), dtype=int32, numpy=0>

0

In [5]:
# 向量
vector = tf.constant([1.0,2.0,3.0])
tf.rank(vector)
vector.numpy().ndim

<tf.Tensor: shape=(), dtype=int32, numpy=1>

1

In [6]:
# 矩阵
matrix = tf.constant([[1,2],[3,4]])
tf.rank(matrix).numpy()
np.ndim(matrix)

2

2

In [7]:
# ３维
t3 = tf.constant([[[1,2],[3,4]],[[1,2],[3,4]]])
t3
tf.rank(t3)

<tf.Tensor: shape=(2, 2, 2), dtype=int32, numpy=
array([[[1, 2],
        [3, 4]],

       [[1, 2],
        [3, 4]]], dtype=int32)>

<tf.Tensor: shape=(), dtype=int32, numpy=3>

In [8]:
t4 = tf.constant([[[[1,2],[3,4]],[[1,2],[3,4]]],[[[5,6],[7,8]],[[5,6],[7,8]]]])
t4

<tf.Tensor: shape=(2, 2, 2, 2), dtype=int32, numpy=
array([[[[1, 2],
         [3, 4]],

        [[1, 2],
         [3, 4]]],


       [[[5, 6],
         [7, 8]],

        [[5, 6],
         [7, 8]]]], dtype=int32)>

可以通过tf.cast改变张量的数据类型

可以用numpy()将tensorflow的张量转换为numpy中的张量

可以用shape()方法查看张量的尺寸

In [9]:
h = tf.constant([11,12],dtype=tf.int32)
f = tf.cast(h,tf.float32)
print(h.dtype,f.dtype)

<dtype: 'int32'> <dtype: 'float32'>


In [10]:
y = tf.constant([[1,2],[3,4]])
y.numpy()
y.shape

array([[1, 2],
       [3, 4]], dtype=int32)

TensorShape([2, 2])

In [11]:
u = tf.constant(u'你好　世界')
print(u.numpy())
print(u.numpy().decode())

b'\xe4\xbd\xa0\xe5\xa5\xbd\xe3\x80\x80\xe4\xb8\x96\xe7\x95\x8c'
你好　世界


## 2.变量张量

模型中被训练的参数一般被设置为变量

In [12]:
c = tf.constant([1,2])
print(c)
print(id(c))

c = c + tf.constant([3,4])
print(c)
print(id(c))
# 常量值不可以改变，重新赋值相当于创造新的内存空间

tf.Tensor([1 2], shape=(2,), dtype=int32)
139666890607136
tf.Tensor([4 6], shape=(2,), dtype=int32)
139666890607640


In [13]:
# 变量的值可以改变，可以通过assign_add等方法重新赋值
v = tf.Variable([1,2],dtype=tf.int8)
print(v)
print(id(v))

v.assign_add([3,4])
print(v)
print(id(v))

<tf.Variable 'Variable:0' shape=(2,) dtype=int8, numpy=array([1, 2], dtype=int8)>
139666890645968


<tf.Variable 'UnreadVariable' shape=(2,) dtype=int8, numpy=array([4, 6], dtype=int8)>

<tf.Variable 'Variable:0' shape=(2,) dtype=int8, numpy=array([4, 6], dtype=int8)>
139666890645968


## 二．三种计算图

存在三种计算图的构建方法：静态计算图，动态计算图，AutoGraph

在TF1.0，采用的是静态计算图，需要首先完成各种算子的创建，然后构建计算图，最后开启一个Session,显式地执行计算图;好处：构建完成之后几乎全部是在TF内部使用C++代码运行，效率更高;此外，还会对计算步骤进行一定的优化．

在TF2.0，采用的是动态计算图，每个算子在定义完毕，自动加入到隐含的默认计算图中立即执行获取到计算结果，无需开启Session.好处是方便调试程序，缺点是运行效率低．

要在Tf2.0使用静态图,可以使用＠tf.function装饰器将普通的python函数转换成对应的TF计算图构建代码，相当于在TF1.0中Session执行代码，称之为AutoGraph.

### １．计算图简介
计算图由节点（nodes）和线(edges)组成．

节点表示操作符Operator,线表示节点间的依赖．

实线表示的是节点之间数据的传递，传递的是张量．

虚线表示的是节点之间控制的依赖，即执行的先后顺序．

![](./../data/strjoin_graph.png)

### 2.静态计算图

In [14]:
## TF2.0兼容性1.0在tf.compact.v1子模块保留对TF1.0计算图的支持
g = tf.compat.v1.Graph()

with g.as_default():
    x = tf.compat.v1.placeholder(name='x',shape=[],dtype=tf.string)
    y = tf.compat.v1.placeholder(name='y',shape=[],dtype=tf.string)
    z = tf.strings.join([x,y],name='join',separator=' ')

with tf.compat.v1.Session(graph=g) as sess:
    ## fetches 相当于函数的返回值
    result = sess.run(fetches=z,feed_dict={x:'hello',y:'world'})
    print(result)

b'hello world'


### 3.动态计算图

In [15]:
## 每个算子都是立即执行
x = tf.constant('hello')
y = tf.constant('world')
z = tf.strings.join([x,y],separator=' ')
tf.print(z)

hello world


In [16]:
## 可以对动态计算图代码的输入和输出关系定义为函数进行封装
def strjoin(x,y,separator=' '):
    z = tf.strings.join([x,y],separator=separator)
    tf.print(z)
    return z
result = strjoin(tf.constant('hello'),tf.constant('world'))
print(result)

hello world
tf.Tensor(b'hello world', shape=(), dtype=string)


### 4.AutoGraph

动态计算图效率较低

可以使用＠tf.function装饰器将普通python函数转换为TF1.0对应的静态图构建代码;

step1: 定义函数

step2: 执行计算图即调用函数

也就是说，可以在动态图中调试代码，需要提升效率的时候加上注解;

In [17]:
import tensorflow as tf
@tf.function
def strjoin(x,y,separator=' '):
    z = tf.strings.join([x,y],separator=separator)
    tf.print(z)
    return z
resu = strjoin(tf.constant('hello'),tf.constant('world'))
print(resu)

hello world
tf.Tensor(b'hello world', shape=(), dtype=string)


In [18]:
import datetime

##开启日志
stamp = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
logdir = './../data/autograph_1/%s' %(stamp)
writer = tf.summary.create_file_writer(logdir=logdir)

##开启AutoGraph跟踪
tf.summary.trace_on(graph=True,profiler=True)

##执行AutoGraph
result = strjoin('hello','world')

#将计算图信息写入到日志
with writer.as_default():
    tf.summary.trace_export(name='autograph',step=0,profiler_outdir=logdir)

hello world


In [19]:
# 启动tensorboard
%load_ext tensorboard

In [20]:
%tensorboard --logdir ./../data/autograph_1/

Reusing TensorBoard on port 6006 (pid 3186), started 0:50:23 ago. (Use '!kill 3186' to kill it.)

## 3.自动微分机制

TF一般使用tf.GradientTape来记录正向运算过程，然后自动获取梯度值．

### 3.1利用梯度磁带求导数

In [21]:
x = tf.Variable(0.,name='x',dtype=tf.float32)
a = tf.constant(1.)
b = tf.constant(-2.)
c = tf.constant(1.)

with tf.GradientTape() as tape:
    y = a * tf.pow(x,2) + b*x + c

dy_dx = tape.gradient(y,x)
print(dy_dx)

tf.Tensor(-2.0, shape=(), dtype=float32)


In [22]:
## 对常量也可以求道，需要增加一个watch
with tf.GradientTape() as tape:
    tape.watch([a,b,c])
    y = a * tf.pow(x,2) + b*x + c
dy_dx,dy_da,dy_db,dy_dc = tape.gradient(y,[x,a,b,c])
print(dy_da)

tf.Tensor(0.0, shape=(), dtype=float32)


In [23]:
## 可以二jie求导
with tf.GradientTape() as out:
    with tf.GradientTape() as tape_in:
        y = a * tf.pow(x,2) + b*x + c
    dy_dx = tape_in.gradient(y,x)
dy2_dx2 = out.gradient(dy_dx,x)
print(dy2_dx2)

tf.Tensor(2.0, shape=(), dtype=float32)


In [25]:
# 可以在AutoGraph中使用

@tf.function
def f(x):
    a = tf.constant(1.)
    b = tf.constant(-2.)
    c = tf.constant(1.)
    
    x = tf.cast(x,tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(x)
        y = a * tf.pow(x,2) + b*x + c
    dy_dx = tape.gradient(y,x)
    return ((dy_dx,y))
tf.print(f(tf.constant(0.0)))  
tf.print(f(tf.constant(1.0)))  

(-2, 1)
(0, 0)


### 3.2利用梯度磁带和优化器求最小值

In [26]:
## 使用optimizer.apply_gradients
x = tf.Variable(0.0,name='x',dtype=tf.float32)
a = tf.constant(1.)
b = tf.constant(-2.)
c = tf.constant(1.)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
for _ in range(1000):
    with tf.GradientTape() as tape:
        y = a * tf.pow(x,2) + b*x + c
    dy_dx = tape.gradient(y,x)
    optimizer.apply_gradients(grads_and_vars=[(dy_dx,x)])
tf.print(y,x)

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=3>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=4>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=5>

...

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=997>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=998>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=999>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1000>

0 0.999998569


In [27]:
# optimizer.minimize相当于先用tape求gradient,再apply_gradients
x = tf.Variable(0.0,name='x',dtype=tf.float32)

def f():
    a = tf.constant(1.)
    b = tf.constant(-2.)
    c = tf.constant(1.)
    y = a * tf.pow(x,2) + b*x + c
    return (y)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
for _ in range(1000):
    optimizer.minimize(f,[x])
tf.print(f(),x)

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=2>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=3>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=4>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=5>

...

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=997>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=998>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=999>

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1000>

0 0.999998569


In [28]:
# 在autograph中完成最小值求解
# 使用optimer.apply_gradients
x = tf.Variable(0.0,name='x',dtype=tf.float32)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

@tf.function
def minimizef():
    a = tf.constant(1.)
    b = tf.constant(-2.)
    c = tf.constant(1.)
    for _ in tf.range(1000):###注意使用的是tf.range(1000)
        with tf.GradientTape() as tape:
            y = a * tf.pow(x,2) + b*x + c
        dy_dx = tape.gradient(y,x)
        optimizer.apply_gradients(grads_and_vars=[(dy_dx,x)])

    y = a * tf.pow(x,2) + b*x + c
    return y
tf.print(minimizef())
tf.print(x)

0
0.999998569


In [29]:
# 在autograph中完成最小值求解
# 使用optimer.minimize
x = tf.Variable(0.0,name='x',dtype=tf.float32)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

def f():
    a = tf.constant(1.)
    b = tf.constant(-2.)
    c = tf.constant(1.)
    y = a * tf.pow(x,2) + b*x + c
    return (y)

@tf.function
def train(epoch):
    for _ in range(epoch):
        optimizer.minimize(f,[x])
    return(f())
tf.print(train(1000))
tf.print(x)

0
0.999998569
